In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pylab import text
import os

# Week X data

In [3]:
##Importing weekly player data

week = 14

players_df = pd.read_csv(f"salaries/Week{week}_salaries_rotowire.csv")
players_df ['dk_salary'] = players_df ['salary'].astype('float')
players_df ['fantasy_score'] = players_df ['proj_points'].astype('float')
players_df ['player'] = players_df ['player']
players_df ['pos'] = players_df ['position']
players_df = players_df[['player','pos','dk_salary','fantasy_score']]


In [4]:
players_df

,player,pos,dk_salary,fantasy_score
0,Tyler Shough,QB,4600.0,17.17


In [5]:
##Plot Points vs Salary, linear trend
x = players_df['dk_salary']
y = players_df['fantasy_score']

z, res, _, _, _ = np.polyfit(x, y, 1, full = True)
p = np.poly1d(z)

##R^2 value
yhat =   [p(z) for z in x]       # or [p(z) for z in x]
ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
r2 = (ssreg / sstot)
print(r2)

#r^2 value another way
SSE = res[0]
diff = y - y.mean()
square_diff = diff **2
SST = square_diff.sum()
r2 = 1 - SSE/SST
print(r2)

fig, ax = plt.subplots(figsize=(20,12))
sns.scatterplot(ax=ax,x='dk_salary',y='fantasy_score',data=players_df,s=70)

plt.title(f'2024 Week {week} points vs salary',fontsize=35)
plt.xlabel('Salary ($)',fontsize=30)
plt.ylabel('Points',fontsize=30)

plt.plot(x,p(x),"r--")

for i,name in enumerate(players_df['player']):
    ax.annotate(name,(x[i],y[i]))

text(0.1, 0.9,'R^2={}'.format('%.3f'%(r2)),fontsize=25,
     horizontalalignment='center',
     verticalalignment='center',
     transform = ax.transAxes)
    
plt.show()

inf


/var/folders/44/11jmwyzd3tz_crgr6qx_9z_w0000gp/T/ipykernel_37981/3466701151.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  r2 = (ssreg / sstot)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
##Seeing which players are above or below trend line
players_df['trend y'] = p(players_df['dk_salary'])
players_df['trend diff'] = players_df['fantasy_score'] - players_df['trend y']

better_players_df = players_df[players_df['trend diff'] > 0]
# better_players_df = better_players_df.sort_values(by=['fantasy_score'],ascending=False)
better_players_df = better_players_df.sort_values(by=['trend diff'],ascending=False)
better_players_df = better_players_df[['player','pos','dk_salary','fantasy_score','trend diff']]

In [7]:
better_players_df

,player,pos,dk_salary,fantasy_score,trend diff
0,Tyler Shough,QB,4600.0,17.17,7.105427e-15


# Make Picks

In [8]:
positions_needed = {
    'QB': 1,
    'RB': 2,
    'WR': 3,
    'TE': 1,
    'DST': 1,
    'FLEX': 1
    }

picks_df = pd.DataFrame(columns = ['player','pos','dk_salary','fantasy_score'])

total_salary = 50000


In [9]:
#Pick best 1 RB, and 1 WR available
best_rb = better_players_df[better_players_df['pos'] == 'RB'].head(1)
best_rb  = best_rb.drop('trend diff',axis=1)
best_wr = better_players_df[better_players_df['pos'] == 'WR'].head(1)
best_wr  = best_wr.drop('trend diff',axis=1)
picks_df = pd.concat([picks_df,best_rb,best_wr])

remaining_salary = total_salary - picks_df['dk_salary'].sum()
remaining_salary

50000

In [10]:
picks_df

,player,pos,dk_salary,fantasy_score


In [11]:
#Best per dollar
better_per_dollar_players_df = better_players_df.copy()
better_per_dollar_players_df['point_per_dollar'] = better_per_dollar_players_df['fantasy_score'] / better_per_dollar_players_df['dk_salary']
better_per_dollar_players_df = better_per_dollar_players_df.sort_values(by=['point_per_dollar'],ascending=False)

In [12]:
better_per_dollar_players_df

,player,pos,dk_salary,fantasy_score,trend diff,point_per_dollar
0,Tyler Shough,QB,4600.0,17.17,7.105427e-15,0.003733


In [13]:
# Pick best value QB
best_qb = better_per_dollar_players_df[better_per_dollar_players_df['pos'] == 'QB'].head(1)
best_qb = best_qb.drop(['point_per_dollar','trend diff'], axis=1)
picks_df = pd.concat([picks_df,best_qb])

remaining_salary = total_salary - picks_df['dk_salary'].sum()
remaining_salary

/var/folders/44/11jmwyzd3tz_crgr6qx_9z_w0000gp/T/ipykernel_37981/3961832111.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  picks_df = pd.concat([picks_df,best_qb])


np.float64(45400.0)

In [14]:
# Pick next best two value RBs
best_rb = better_per_dollar_players_df[better_per_dollar_players_df['pos'] == 'RB']
best_rb  = best_rb.drop(['point_per_dollar','trend diff'], axis=1)
for i in range(2):
    for index, row in best_rb.iterrows():
        if row['player'] not in picks_df['player'].values:
            print(row['player'])
            picks_df = pd.concat([picks_df,row.to_frame().T])
            break

In [15]:
picks_df

,player,pos,dk_salary,fantasy_score
0,Tyler Shough,QB,4600.0,17.17


In [16]:
total_salary - picks_df['dk_salary'].sum()

np.float64(45400.0)

In [17]:
# Pick next best two value WRs that cost 1/4 or less than remaining salary
best_wr = better_per_dollar_players_df[better_per_dollar_players_df['pos'] == 'WR']
best_wr  = best_wr.drop(['point_per_dollar','trend diff'], axis=1)
remaining_salary = total_salary - picks_df['dk_salary'].sum()
for i in range(2):
    for index, row in best_wr.iterrows():
        if row['player'] not in picks_df['player'].values and row['dk_salary'] <= remaining_salary/4:
            print(row['player'])
            picks_df = pd.concat([picks_df,row.to_frame().T])
            break

In [18]:
#Pick best DST if possible
best_dst = better_per_dollar_players_df[better_per_dollar_players_df['pos'] ==  'D'].head(1)
best_dst = best_dst.drop(['trend diff','point_per_dollar'],axis=1)
picks_df = pd.concat([picks_df,best_dst])

In [19]:
best_dst

,player,pos,dk_salary,fantasy_score


In [20]:
#Pick best TE I can afford
best_te = better_per_dollar_players_df[better_per_dollar_players_df['pos'] ==  'TE']
best_te = best_te.drop(['point_per_dollar','trend diff'], axis=1)
remaining_salary = total_salary - picks_df['dk_salary'].sum()
for index, row in best_te.iterrows():
    if row['dk_salary'] <= remaining_salary:
        print(row['player'])
        picks_df = pd.concat([picks_df,row.to_frame().T])
        break

In [21]:
best_te

,player,pos,dk_salary,fantasy_score


In [22]:
picks_df

,player,pos,dk_salary,fantasy_score
0,Tyler Shough,QB,4600.0,17.17


In [23]:
total_salary - picks_df['dk_salary'].sum()

np.float64(45400.0)

In [24]:
# Use remaining salary to reinvest into most expensive QB
pick_qb_price = picks_df[picks_df['pos'] == 'QB']['dk_salary'].values[0]
remaining_salary = total_salary - picks_df['dk_salary'].sum()

# pick_qb_price
qbs = better_players_df[better_players_df['pos'] == 'QB']
new_qb = qbs[qbs['dk_salary'] <= remaining_salary + pick_qb_price].sort_values('dk_salary',ascending=False).head(1)
new_qb = new_qb.drop('trend diff',axis=1)

# remove old qb add new qb
picks_df = picks_df[picks_df['pos'] != 'QB']
picks_df = pd.concat([picks_df,new_qb])


In [25]:
picks_df

,player,pos,dk_salary,fantasy_score
0,Tyler Shough,QB,4600.0,17.17


In [26]:
total_salary - picks_df['dk_salary'].sum()

np.float64(45400.0)

In [27]:
# Use remaining salary to replace cheapest WR with more expensive WR
pick_wr_price = picks_df[picks_df['pos'] == 'WR']['dk_salary'].values.min()
remaining_salary = total_salary - picks_df['dk_salary'].sum()

# pick_wr_price
wrs = better_players_df[better_players_df['pos'] == 'WR']
new_wr = wrs[wrs['dk_salary'] <= remaining_salary + pick_wr_price].sort_values('dk_salary',ascending=False).head(1)
new_wr = new_wr.drop('trend diff',axis=1)

old_wr = picks_df[picks_df['dk_salary'] == pick_wr_price]
print(old_wr)
print(new_wr)

# remove old wr add new wr
picks_df = picks_df.drop(old_wr.index)
picks_df = pd.concat([picks_df,new_wr])

ValueError: zero-size array to reduction operation minimum which has no identity

In [28]:
picks_df = picks_df.drop('fantasy_score',axis=1)
picks_df = picks_df.rename(columns={'dk_salary':'salary'})

In [29]:
picks_df

,player,pos,salary
0,Tyler Shough,QB,4600.0


In [29]:
picks_df.to_csv(f'picks/antonio_trani_week{week}_picks.csv',index=False)

In [31]:
picks_df['salary'].sum()

50000.0